### Questions tagging with NLP: Case of StackOvrFlow

In [181]:
# Importing librairies
import pandas as pd
import nltk
import numpy as np
from bs4 import BeautifulSoup
import stop_words
import re
import spacy
from gensim.models import Phrases, phrases
from gensim import models, corpora
import pyLDAvis.gensim
from gensim.models.coherencemodel import CoherenceModel

### Importing necessary tools

In [86]:
# Loading CSV File
data = pd.read_csv('QueryResults.csv')
data.head(3)

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,...,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate
0,4,1,7.0,NaN,2008-07-31 21:42:52,NaN,564,36447.0,"<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n",8.0,...,Rich B,2018-07-02 17:55:27,2018-07-02 17:55:27,Convert Decimal to Double?,<c#><floating-point><type-conversion><double><decimal>,13.0,2,41.0,NaN,2012-10-31 16:42:47
1,6,1,31.0,NaN,2008-07-31 22:08:08,NaN,253,16153.0,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n",9.0,...,Rich B,2016-03-19 06:05:48,2016-03-19 06:10:52,Percentage width child element in absolutely positioned parent on Internet Explorer 7,<html><css><css3><internet-explorer-7>,5.0,0,10.0,NaN,NaN
2,7,2,NaN,4.0,2008-07-31 22:17:57,NaN,398,NaN,<p>An explicit cast to double like this isn't necessary:</p>\n\n<pre><code>double trans = (double) trackBar1.Value / 5000.0;\n</code></pre>\n\n<p>Identifying the constant as <code>5000.0</code> (or as <code>5000d</code>) is sufficient:</p>\n\n<pre><code>double trans = trackBar1.Value / 5000.0;\ndouble trans = trackBar1.Value / 5000d;\n</code></pre>\n,9.0,...,NaN,2017-12-16 05:06:57,2017-12-16 05:06:57,NaN,NaN,NaN,0,NaN,NaN,NaN


In [87]:
data.shape

(27412, 22)

In [88]:
# Percentage of missing data by variable
data.isnull().sum().sort_values(ascending=False)*100/data.shape[0]

DeletionDate             100.000000
ClosedDate               97.336933 
CommunityOwnedDate       94.786955 
FavoriteCount            85.568364 
AcceptedAnswerId         83.025682 
ViewCount                79.855538 
AnswerCount              79.855538 
Tags                     79.855538 
Title                    79.855538 
LastEditorDisplayName    76.444623 
LastEditorUserId         59.787684 
LastEditDate             59.357216 
ParentId                 20.144462 
OwnerDisplayName         15.828834 
OwnerUserId              1.926164  
Body                     0.000000  
Score                    0.000000  
CreationDate             0.000000  
LastActivityDate         0.000000  
CommentCount             0.000000  
PostTypeId               0.000000  
Id                       0.000000  
dtype: float64

Around 80% of Tags and Title are missing values. Body have any missing values. In this case, we think already to the semi-unsupervised machine learning.

In [89]:
data_imp = data[['Body','Title','Tags']]
data_imp.head()

,Body,Title,Tags
0,"<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n",Convert Decimal to Double?,<c#><floating-point><type-conversion><double><decimal>
1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n",Percentage width child element in absolutely positioned parent on Internet Explorer 7,<html><css><css3><internet-explorer-7>
2,<p>An explicit cast to double like this isn't necessary:</p>\n\n<pre><code>double trans = (double) trackBar1.Value / 5000.0;\n</code></pre>\n\n<p>Identifying the constant as <code>5000.0</code> (or as <code>5000d</code>) is sufficient:</p>\n\n<pre><code>double trans = trackBar1.Value / 5000.0;\ndouble trans = trackBar1.Value / 5000d;\n</code></pre>\n,NaN,NaN
3,"<p>Given a <code>DateTime</code> representing a person's birthday, how do I calculate their age in years? </p>\n",How do I calculate someone's age in C#?,<c#><.net><datetime>
4,"<p>Given a specific <code>DateTime</code> value, how do I display relative time, like:</p>\n\n<ul>\n<li>2 hours ago</li>\n<li>3 days ago</li>\n<li>a month ago</li>\n</ul>\n",Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-span>


In [90]:
# #delete all missing tags values
# data_imp1 = data_imp.dropna(axis=0, subset="Tags")

### Tokenization 

In [91]:
def reg_function(x):
    tokenizer = nltk.RegexpTokenizer(r'<.*?>')
    if x == x:
        x = tokenizer.tokenize(x.lower())
        for tag in range(len(x)):
            x[tag] = x[tag].strip('>').strip('<')
    return x

In [92]:
data_imp['token_tag'] = data_imp['Tags'].apply(reg_function)
data_imp['token_tag'].head()

C:\Users\mdegni\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0    [c#, floating-point, type-conversion, double, decimal]
1    [html, css, css3, internet-explorer-7]                
2    NaN                                                   
3    [c#, .net, datetime]                                  
4    [c#, datetime, time, datediff, relative-time-span]    
Name: token_tag, dtype: object

In [93]:
data_imp.head()

,Body,Title,Tags,token_tag
0,"<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n",Convert Decimal to Double?,<c#><floating-point><type-conversion><double><decimal>,"[c#, floating-point, type-conversion, double, decimal]"
1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n",Percentage width child element in absolutely positioned parent on Internet Explorer 7,<html><css><css3><internet-explorer-7>,"[html, css, css3, internet-explorer-7]"
2,<p>An explicit cast to double like this isn't necessary:</p>\n\n<pre><code>double trans = (double) trackBar1.Value / 5000.0;\n</code></pre>\n\n<p>Identifying the constant as <code>5000.0</code> (or as <code>5000d</code>) is sufficient:</p>\n\n<pre><code>double trans = trackBar1.Value / 5000.0;\ndouble trans = trackBar1.Value / 5000d;\n</code></pre>\n,NaN,NaN,NaN
3,"<p>Given a <code>DateTime</code> representing a person's birthday, how do I calculate their age in years? </p>\n",How do I calculate someone's age in C#?,<c#><.net><datetime>,"[c#, .net, datetime]"
4,"<p>Given a specific <code>DateTime</code> value, how do I display relative time, like:</p>\n\n<ul>\n<li>2 hours ago</li>\n<li>3 days ago</li>\n<li>a month ago</li>\n</ul>\n",Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-span>,"[c#, datetime, time, datediff, relative-time-span]"


In [94]:
# EDA of tag
data_imp1 = data_imp.dropna(subset=['Tags'])
data_imp1.head()

,Body,Title,Tags,token_tag
0,"<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n",Convert Decimal to Double?,<c#><floating-point><type-conversion><double><decimal>,"[c#, floating-point, type-conversion, double, decimal]"
1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n",Percentage width child element in absolutely positioned parent on Internet Explorer 7,<html><css><css3><internet-explorer-7>,"[html, css, css3, internet-explorer-7]"
3,"<p>Given a <code>DateTime</code> representing a person's birthday, how do I calculate their age in years? </p>\n",How do I calculate someone's age in C#?,<c#><.net><datetime>,"[c#, .net, datetime]"
4,"<p>Given a specific <code>DateTime</code> value, how do I display relative time, like:</p>\n\n<ul>\n<li>2 hours ago</li>\n<li>3 days ago</li>\n<li>a month ago</li>\n</ul>\n",Calculate relative time in C#,<c#><datetime><time><datediff><relative-time-span>,"[c#, datetime, time, datediff, relative-time-span]"
6,<p>Is there any standard way for a Web Server to be able to determine a user's timezone within a web page? </p>\n\n<p>Perhaps from an HTTP header or part of the user-agent string?</p>\n,Determine a User's Timezone,<javascript><html><browser><timezone><timezoneoffset>,"[javascript, html, browser, timezone, timezoneoffset]"


In [95]:
# Occurrence of Tags
# 1. Define a list of token values from dataframe
list_tok = data_imp1.token_tag.values
list_tok


array([list(['c#', 'floating-point', 'type-conversion', 'double', 'decimal']),
       list(['html', 'css', 'css3', 'internet-explorer-7']),
       list(['c#', '.net', 'datetime']), ..., list(['html', 'css']),
       list(['.net', 'dll']), list(['windows'])], dtype=object)

In [96]:
# 2. Put all element in one list
list_tag = []
for sublist in list_tok:
    for item in sublist:
        list_tag.append(item)

In [97]:
sorted(list_tag)

['.htaccess',
 '.htaccess',
 '.htaccess',
 '.htaccess',
 '.htpasswd',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.net',
 '.

In [98]:
# Create a tuple with number of occurence of unique tag in the list
from itertools import groupby
freq = {key:len(list(group)) for key, group in groupby(np.sort(list_tag))} 

In [99]:
freq

{'.htaccess': 4,
 '.htpasswd': 1,
 '.net': 628,
 '.net-1.0': 1,
 '.net-1.1': 9,
 '.net-2.0': 29,
 '.net-3.5': 54,
 '.net-assembly': 1,
 '.net-attributes': 2,
 '.net-client-profile': 3,
 '2d': 1,
 '2d-games': 1,
 '32-bit': 3,
 '32bit-64bit': 1,
 '3d': 2,
 '3d-engine': 1,
 '3des': 1,
 '64bit': 15,
 'abap': 1,
 'abstract-class': 1,
 'abstraction': 2,
 'access-specifier': 1,
 'accessibility': 2,
 'account': 2,
 'accurev': 1,
 'acl': 1,
 'acrobat': 1,
 'action': 1,
 'actionlistener': 1,
 'actionscript': 8,
 'actionscript-2': 1,
 'actionscript-3': 36,
 'activation': 2,
 'active-directory': 13,
 'active-directory-group': 1,
 'activemq': 3,
 'activerecord': 7,
 'activereports': 1,
 'activex': 5,
 'adam': 2,
 'adc': 1,
 'add-in': 5,
 'add-on': 3,
 'addclass': 1,
 'address-bar': 1,
 'administration': 6,
 'ado': 1,
 'ado.net': 16,
 'adobe': 10,
 'adobe-reader': 1,
 'adsl': 1,
 'advanced-queuing': 1,
 'agent-based-modeling': 1,
 'aggregation': 1,
 'agile': 12,
 'air': 12,
 'aix': 1,
 'ajax': 45,
 

In [100]:
X_tag = np.array(freq)
X_tag

array({'.htaccess': 4, '.htpasswd': 1, '.net': 628, '.net-1.0': 1, '.net-1.1': 9, '.net-2.0': 29, '.net-3.5': 54, '.net-assembly': 1, '.net-attributes': 2, '.net-client-profile': 3, '2d': 1, '2d-games': 1, '32-bit': 3, '32bit-64bit': 1, '3d': 2, '3d-engine': 1, '3des': 1, '64bit': 15, 'abap': 1, 'abstract-class': 1, 'abstraction': 2, 'access-specifier': 1, 'accessibility': 2, 'account': 2, 'accurev': 1, 'acl': 1, 'acrobat': 1, 'action': 1, 'actionlistener': 1, 'actionscript': 8, 'actionscript-2': 1, 'actionscript-3': 36, 'activation': 2, 'active-directory': 13, 'active-directory-group': 1, 'activemq': 3, 'activerecord': 7, 'activereports': 1, 'activex': 5, 'adam': 2, 'adc': 1, 'add-in': 5, 'add-on': 3, 'addclass': 1, 'address-bar': 1, 'administration': 6, 'ado': 1, 'ado.net': 16, 'adobe': 10, 'adobe-reader': 1, 'adsl': 1, 'advanced-queuing': 1, 'agent-based-modeling': 1, 'aggregation': 1, 'agile': 12, 'air': 12, 'aix': 1, 'ajax': 45, 'alert': 2, 'algebraic-data-types': 1, 'algorithm': 

In [101]:
import matplotlib.pyplot as plt

In [102]:
X_tag

array({'.htaccess': 4, '.htpasswd': 1, '.net': 628, '.net-1.0': 1, '.net-1.1': 9, '.net-2.0': 29, '.net-3.5': 54, '.net-assembly': 1, '.net-attributes': 2, '.net-client-profile': 3, '2d': 1, '2d-games': 1, '32-bit': 3, '32bit-64bit': 1, '3d': 2, '3d-engine': 1, '3des': 1, '64bit': 15, 'abap': 1, 'abstract-class': 1, 'abstraction': 2, 'access-specifier': 1, 'accessibility': 2, 'account': 2, 'accurev': 1, 'acl': 1, 'acrobat': 1, 'action': 1, 'actionlistener': 1, 'actionscript': 8, 'actionscript-2': 1, 'actionscript-3': 36, 'activation': 2, 'active-directory': 13, 'active-directory-group': 1, 'activemq': 3, 'activerecord': 7, 'activereports': 1, 'activex': 5, 'adam': 2, 'adc': 1, 'add-in': 5, 'add-on': 3, 'addclass': 1, 'address-bar': 1, 'administration': 6, 'ado': 1, 'ado.net': 16, 'adobe': 10, 'adobe-reader': 1, 'adsl': 1, 'advanced-queuing': 1, 'agent-based-modeling': 1, 'aggregation': 1, 'agile': 12, 'air': 12, 'aix': 1, 'ajax': 45, 'alert': 2, 'algebraic-data-types': 1, 'algorithm': 

### Construct plot

### Dealing with Body

In [71]:
pd.set_option('display.max_colwidth', -1)

In [72]:
from tqdm import tqdm, tqdm_notebook # progress bars in Jupyter
tqdm.pandas()
tqdm_notebook()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

0it [00:00, ?it/s]

In [73]:
data_imp[['Body']].head()

,Body
0,"<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n"
1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n"
2,<p>An explicit cast to double like this isn't necessary:</p>\n\n<pre><code>double trans = (double) trackBar1.Value / 5000.0;\n</code></pre>\n\n<p>Identifying the constant as <code>5000.0</code> (or as <code>5000d</code>) is sufficient:</p>\n\n<pre><code>double trans = trackBar1.Value / 5000.0;\ndouble trans = trackBar1.Value / 5000d;\n</code></pre>\n
3,"<p>Given a <code>DateTime</code> representing a person's birthday, how do I calculate their age in years? </p>\n"
4,"<p>Given a specific <code>DateTime</code> value, how do I display relative time, like:</p>\n\n<ul>\n<li>2 hours ago</li>\n<li>3 days ago</li>\n<li>a month ago</li>\n</ul>\n"


In [130]:
# Cleaning text
def get_text(x):
    x = BeautifulSoup(x, 'lxml').get_text() # Get text in htlm with beautifulsoup
    tokenizer = nltk.RegexpTokenizer(r'\w+') # Use regular expression to delete \n
    x = tokenizer.tokenize(x.lower()) # apply on x by tranforming x in lower character
    return x

In [131]:
data_imp['token_body'] = data_imp['Body'].apply(get_text)
data_imp[['token_body']].head()

C:\Users\mdegni\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,token_body
0,"[i, want, to, use, a, track, bar, to, change, a, form, s, opacity, this, is, my, code, decimal, trans, trackbar1, value, 5000, this, opacity, trans, when, i, build, the, application, it, gives, the, following, error, cannot, implicitly, convert, type, decimal, to, double, i, tried, using, trans, and, double, but, then, the, control, doesn, t, work, this, code, worked, fine, in, a, past, vb, net, project]"
1,"[i, have, an, absolutely, positioned, div, containing, several, children, one, of, which, is, a, relatively, positioned, div, when, i, use, a, percentage, based, width, on, the, child, div, it, collapses, to, 0, width, on, internet, explorer, 7, but, not, on, firefox, or, safari, if, i, use, pixel, width, it, works, if, the, parent, is, relatively, positioned, the, percentage, width, on, the, child, works, is, there, something, i, m, missing, here, is, there, an, easy, fix, for, this, besides, the, pixel, based, width, on, the, child, is, there, an, area, of, the, css, specification, that, covers, this]"
2,"[an, explicit, cast, to, double, like, this, isn, t, necessary, double, trans, double, trackbar1, value, 5000, 0, identifying, the, constant, as, 5000, 0, or, as, 5000d, is, sufficient, double, trans, trackbar1, value, 5000, 0, double, trans, trackbar1, value, 5000d]"
3,"[given, a, datetime, representing, a, person, s, birthday, how, do, i, calculate, their, age, in, years]"
4,"[given, a, specific, datetime, value, how, do, i, display, relative, time, like, 2, hours, ago, 3, days, ago, a, month, ago]"


In [134]:
# Define stopwords
sw = stop_words.get_stop_words(language='en')

In [139]:
# Delete stopwords with 1-gram
data_imp['tokens_clean'] = data_imp.token_body.map(
    lambda tok: [t.lower() for t in re.split(" ", re.sub(r"(\W+|_|\d+)", " ", " ".join(tok)))
                 if t.lower() not in sw and len(t) > 1]
)

C:\Users\mdegni\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [140]:
data_imp[['tokens_clean']].head()

,tokens_clean
0,"[want, use, track, bar, change, form, opacity, code, decimal, trans, trackbar, value, opacity, trans, build, application, gives, following, error, implicitly, convert, type, decimal, double, tried, using, trans, double, control, doesn, work, code, worked, fine, past, vb, net, project]"
1,"[absolutely, positioned, div, containing, several, children, one, relatively, positioned, div, use, percentage, based, width, child, div, collapses, width, internet, explorer, firefox, safari, use, pixel, width, works, parent, relatively, positioned, percentage, width, child, works, something, missing, easy, fix, besides, pixel, based, width, child, area, css, specification, covers]"
2,"[explicit, cast, double, like, isn, necessary, double, trans, double, trackbar, value, identifying, constant, sufficient, double, trans, trackbar, value, double, trans, trackbar, value]"
3,"[given, datetime, representing, person, birthday, calculate, age, years]"
4,"[given, specific, datetime, value, display, relative, time, like, hours, ago, days, ago, month, ago]"


In [154]:
# Lemmatization
lem = spacy.load('en_core_web_md')
data_imp['tokens_clean_lemma'] = data_imp.tokens_clean.progress_map(
    lambda x: [tok.lemma_ for tok in lem(' '.join(x))])
data_imp.head()










  0%|                                                                                        | 0/27412 [00:00<?, ?it/s]








  0%|                                                                              | 2/27412 [00:00<2:35:15,  2.94it/s]








  0%|                                                                              | 7/27412 [00:00<1:51:34,  4.09it/s]








  0%|                                                                             | 12/27412 [00:00<1:21:54,  5.58it/s]








  0%|                                                                             | 15/27412 [00:01<1:04:03,  7.13it/s]








  0%|                                                                               | 19/27412 [00:01<49:10,  9.28it/s]








  0%|                                                                               | 25/27412 [00:01<37:18, 12.24it/s]








  0%|                                                                               | 29/27412 

  1%|▊                                                                             | 268/27412 [00:09<13:28, 33.56it/s]








  1%|▊                                                                             | 272/27412 [00:09<13:19, 33.93it/s]








  1%|▊                                                                             | 276/27412 [00:09<12:50, 35.20it/s]








  1%|▊                                                                             | 281/27412 [00:09<11:55, 37.94it/s]








  1%|▊                                                                             | 285/27412 [00:09<12:53, 35.06it/s]








  1%|▊                                                                             | 289/27412 [00:09<12:32, 36.04it/s]








  1%|▊                                                                             | 295/27412 [00:10<11:36, 38.92it/s]








  1%|▊                                                                             | 300/27412 [00:10<11

  2%|█▋                                                                            | 585/27412 [00:17<11:30, 38.86it/s]








  2%|█▋                                                                            | 592/27412 [00:17<10:23, 42.98it/s]








  2%|█▋                                                                            | 597/27412 [00:17<10:34, 42.26it/s]








  2%|█▋                                                                            | 602/27412 [00:17<12:33, 35.56it/s]








  2%|█▋                                                                            | 606/27412 [00:17<12:17, 36.35it/s]








  2%|█▋                                                                            | 612/27412 [00:18<11:26, 39.03it/s]








  2%|█▊                                                                            | 618/27412 [00:18<10:38, 41.96it/s]








  2%|█▊                                                                            | 623/27412 [00:18<11

  3%|██▌                                                                           | 919/27412 [00:25<10:09, 43.50it/s]








  3%|██▋                                                                           | 924/27412 [00:25<10:30, 41.99it/s]








  3%|██▋                                                                           | 929/27412 [00:25<10:40, 41.32it/s]








  3%|██▋                                                                           | 935/27412 [00:25<10:16, 42.98it/s]








  3%|██▋                                                                           | 940/27412 [00:25<10:06, 43.62it/s]








  3%|██▋                                                                           | 945/27412 [00:25<09:48, 44.97it/s]








  3%|██▋                                                                           | 950/27412 [00:26<09:47, 45.07it/s]








  3%|██▋                                                                           | 955/27412 [00:26<09

  5%|███▌                                                                         | 1255/27412 [00:33<12:57, 33.66it/s]








  5%|███▌                                                                         | 1259/27412 [00:33<13:49, 31.54it/s]








  5%|███▌                                                                         | 1264/27412 [00:33<13:19, 32.69it/s]








  5%|███▌                                                                         | 1272/27412 [00:34<11:12, 38.87it/s]








  5%|███▌                                                                         | 1277/27412 [00:34<10:52, 40.05it/s]








  5%|███▌                                                                         | 1282/27412 [00:34<11:01, 39.51it/s]








  5%|███▌                                                                         | 1287/27412 [00:34<11:48, 36.85it/s]








  5%|███▋                                                                         | 1291/27412 [00:34<12

...

 94%|███████████████████████████████████████████████████████████████████████▊    | 25890/27412 [09:40<00:25, 58.74it/s]








 94%|███████████████████████████████████████████████████████████████████████▊    | 25897/27412 [09:40<00:24, 61.67it/s]








 94%|███████████████████████████████████████████████████████████████████████▊    | 25904/27412 [09:40<00:23, 63.90it/s]








 95%|███████████████████████████████████████████████████████████████████████▊    | 25911/27412 [09:40<00:22, 65.57it/s]








 95%|███████████████████████████████████████████████████████████████████████▊    | 25918/27412 [09:40<00:24, 61.12it/s]








 95%|███████████████████████████████████████████████████████████████████████▉    | 25925/27412 [09:40<00:26, 56.22it/s]








 95%|███████████████████████████████████████████████████████████████████████▉    | 25931/27412 [09:40<00:27, 54.21it/s]








 95%|███████████████████████████████████████████████████████████████████████▉    | 25937/27412 [09:40<00

 96%|████████████████████████████████████████████████████████████████████████▉   | 26298/27412 [09:48<00:23, 47.22it/s]








 96%|████████████████████████████████████████████████████████████████████████▉   | 26303/27412 [09:48<00:24, 45.93it/s]








 96%|████████████████████████████████████████████████████████████████████████▉   | 26309/27412 [09:48<00:24, 45.75it/s]








 96%|████████████████████████████████████████████████████████████████████████▉   | 26316/27412 [09:48<00:22, 48.49it/s]








 96%|████████████████████████████████████████████████████████████████████████▉   | 26321/27412 [09:48<00:23, 45.81it/s]








 96%|████████████████████████████████████████████████████████████████████████▉   | 26328/27412 [09:48<00:24, 45.14it/s]








 96%|█████████████████████████████████████████████████████████████████████████   | 26336/27412 [09:48<00:20, 51.60it/s]








 96%|█████████████████████████████████████████████████████████████████████████   | 26342/27412 [09:48<00

 97%|██████████████████████████████████████████████████████████████████████████  | 26708/27412 [09:56<00:15, 45.91it/s]








 97%|██████████████████████████████████████████████████████████████████████████  | 26716/27412 [09:56<00:13, 51.04it/s]








 97%|██████████████████████████████████████████████████████████████████████████  | 26722/27412 [09:56<00:14, 47.78it/s]








 98%|██████████████████████████████████████████████████████████████████████████  | 26728/27412 [09:56<00:14, 47.84it/s]








 98%|██████████████████████████████████████████████████████████████████████████▏ | 26737/27412 [09:56<00:12, 55.63it/s]








 98%|██████████████████████████████████████████████████████████████████████████▏ | 26744/27412 [09:56<00:11, 56.03it/s]








 98%|██████████████████████████████████████████████████████████████████████████▏ | 26752/27412 [09:56<00:11, 59.41it/s]








 98%|██████████████████████████████████████████████████████████████████████████▏ | 26759/27412 [09:57<00

 99%|███████████████████████████████████████████████████████████████████████████▎| 27146/27412 [10:03<00:04, 61.19it/s]








 99%|███████████████████████████████████████████████████████████████████████████▎| 27154/27412 [10:04<00:03, 65.78it/s]








 99%|███████████████████████████████████████████████████████████████████████████▎| 27161/27412 [10:04<00:04, 60.72it/s]








 99%|███████████████████████████████████████████████████████████████████████████▎| 27168/27412 [10:04<00:03, 61.04it/s]








 99%|███████████████████████████████████████████████████████████████████████████▎| 27175/27412 [10:04<00:04, 53.70it/s]








 99%|███████████████████████████████████████████████████████████████████████████▎| 27181/27412 [10:04<00:04, 55.17it/s]








 99%|███████████████████████████████████████████████████████████████████████████▍| 27189/27412 [10:04<00:03, 59.44it/s]








 99%|███████████████████████████████████████████████████████████████████████████▍| 27196/27412 [10:04<00

,Body,Title,Tags,token_tag,token_body,tokens_clean,tokens_clean_lemma
0,"<p>I want to use a track-bar to change a form's opacity.</p>\n\n<p>This is my code:</p>\n\n<pre><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n\n<p>When I build the application, it gives the following error:</p>\n\n<blockquote>\n <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p>\n</blockquote>\n\n<p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>\n",Convert Decimal to Double?,<c#><floating-point><type-conversion><double><decimal>,"[c#, floating-point, type-conversion, double, decimal]","[i, want, to, use, a, track, bar, to, change, a, form, s, opacity, this, is, my, code, decimal, trans, trackbar1, value, 5000, this, opacity, trans, when, i, build, the, application, it, gives, the, following, error, cannot, implicitly, convert, type, decimal, to, double, i, tried, using, trans, and, double, but, then, the, control, doesn, t, work, this, code, worked, fine, in, a, past, vb, net, project]","[want, use, track, bar, change, form, opacity, code, decimal, trans, trackbar, value, opacity, trans, build, application, gives, following, error, implicitly, convert, type, decimal, double, tried, using, trans, double, control, doesn, work, code, worked, fine, past, vb, net, project]","[want, use, track, bar, change, form, opacity, code, decimal, tran, trackbar, value, opacity, tran, build, application, give, follow, error, implicitly, convert, type, decimal, double, try, use, tran, double, control, doesn, work, code, work, fine, past, vb, net, project]"
1,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p>\n\n<p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p>\n\n<ol>\n<li>Is there something I'm missing here?</li>\n<li>Is there an easy fix for this besides the <em>pixel-based width</em> on the\nchild?</li>\n<li>Is there an area of the CSS specification that covers this?</li>\n</ol>\n",Percentage width child element in absolutely positioned parent on Internet Explorer 7,<html><css><css3><internet-explorer-7>,"[html, css, css3, internet-explorer-7]","[i, have, an, absolutely, positioned, div, containing, several, children, one, of, which, is, a, relatively, positioned, div, when, i, use, a, percentage, based, width, on, the, child, div, it, collapses, to, 0, width, on, internet, explorer, 7, but, not, on, firefox, or, safari, if, i, use, pixel, width, it, works, if, the, parent, is, relatively, positioned, the, percentage, width, on, the, child, works, is, there, something, i, m, missing, here, is, there, an, easy, fix, for, this, besides, the, pixel, based, width, on, the, child, is, there, an, area, of, the, css, specification, that, covers, this]","[absolutely, positioned, div, containing, several, children, one, relatively, positioned, div, use, percentage, based, width, child, div, collapses, width, internet, explorer, firefox, safari, use, pixel, width, works, parent, relatively, positioned, percentage, width, child, works, something, missing, easy, fix, besides, pixel, based, width, child, area, css, specification, covers]","[absolutely, position, div, contain, several, child, one, relatively, position, div, use, percentage, base, width, child, div, collapse, width, internet, explorer, firefox, safari, use, pixel, width, work, parent, relatively, position, percentage, width, child, work, something, miss, easy, fix, besides, pixel, base, width, child, area, css, specific

### Build corpus with Back of Word

In [160]:
# Use bigram and trigram to catch combination of 2/3 words that have a specific meaning together
tokens_lemma = data_imp['tokens_clean_lemma'].tolist()
bigram_lemma =  Phrases(tokens_lemma, min_count = 5)
trigram_lemma = Phrases(bigram_lemma[tokens_lemma])
# for faster implementation
bigram_mod_lemma = phrases.Phraser(bigram_lemma)
trigram_mod_lemma = phrases.Phraser(trigram_lemma)

list(trigram_mod_lemma[bigram_mod_lemma[tokens_lemma]])
tokens_lemma = [[token.lower() for token in t if token.lower() not in sw] for t in tokens_lemma]
tokens_lemma

C:\Users\mdegni\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['want',
  'use',
  'track',
  'bar',
  'change',
  'form',
  'opacity',
  'code',
  'decimal',
  'tran',
  'trackbar_value',
  'opacity',
  'tran',
  'build',
  'application',
  'give',
  'follow_error',
  'implicitly',
  'convert',
  'type',
  'decimal',
  'double',
  'try',
  'use',
  'tran',
  'double',
  'control',
  'doesn_work',
  'code',
  'work_fine',
  'past',
  'vb_net',
  'project'],
 ['absolutely_position_div',
  'contain',
  'several',
  'child',
  'one',
  'relatively',
  'position_div',
  'use',
  'percentage',
  'base',
  'width',
  'child',
  'div',
  'collapse',
  'width',
  'internet_explorer',
  'firefox_safari',
  'use',
  'pixel',
  'width',
  'work',
  'parent',
  'relatively',
  'position',
  'percentage',
  'width',
  'child',
  'work',
  'something',
  'miss',
  'easy',
  'fix',
  'besides',
  'pixel',
  'base',
  'width',
  'child',
  'area',
  'css',
  'specification',
  'cover'],
 ['explicit',
  'cast',
  'double',
  'like',
  'isn',
  'necessary',
  'dou

### Training unsupervised model (LDA)

In [163]:
# dictionary
dictionary_LDA_lemma = corpora.Dictionary(tokens_lemma)
dictionary_LDA_lemma.filter_extremes(no_below=5, no_above=0.9)
corpus_lemma = [dictionary_LDA_lemma.doc2bow(tok) for tok in tokens_lemma]
print(dictionary_LDA_lemma)

Dictionary(13249 unique tokens: ['application', 'bar', 'build', 'change', 'code']...)


In [183]:
num_topics = 30
%time lda_model_lemma = models.LdaModel(corpus_lemma, num_topics=num_topics, \
                                  id2word=dictionary_LDA_lemma, \
                                  passes=10, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA_lemma.keys()))

C:\Users\mdegni\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Wall time: 4min 13s


In [184]:
for i,topic in lda_model_lemma.show_topics(formatted=True, num_topics=3, num_words=5):
    print(str(i)+": "+ topic)
    print()

19: 0.052*"library" + 0.047*"use" + 0.031*"support" + 0.022*"include" + 0.019*"api"

20: 0.065*"com" + 0.065*"http" + 0.041*"http_www" + 0.034*"url" + 0.025*"html"

6: 0.077*"event" + 0.072*"thread" + 0.036*"service" + 0.025*"document" + 0.025*"date"



### Visualization

In [185]:
pyLDAvis.enable_notebook()
vis_lemma = pyLDAvis.gensim.prepare(lda_model_lemma, corpus_lemma, dictionary_LDA_lemma)
vis_lemma

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      15.222616  1        1      -0.180734  0.072317
18     10.690162  1        2      -0.182850  0.032958
11     5.120811   1        3      -0.161767  0.150945
9      5.001297   1        4      -0.174569 -0.009445
5      4.747772   1        5      -0.065377 -0.137011
0      4.295104   1        6      -0.114540 -0.022896
17     4.146711   1        7      -0.155870  0.001898
4      4.123816   1        8      -0.163167  0.007532
29     4.063622   1        9      -0.063385 -0.146074
7      3.505295   1        10     -0.031592 -0.189378
21     3.132708   1        11     -0.091377 -0.029819
8      2.998035   1        12     -0.110190 -0.011866
19     2.832781   1        13     -0.070763  0.210026
25     2.700058   1        14      0.157179 -0.213275
24     2.431106   1        15      0.010724 -0.107726
14     2.171285   1        16      0.001765 -0.063080
23     2.130498   1        17      0.046830 -0.056444
16     2.096958   1        18      0.002977 -0.016259
10     2.003140   1        19     -0.007466  0.064355
20     1.889129   1        20      0.109846  0.121737
1      1.885769   1        21     -0.012655  0.106476
26     1.857666   1        22      0.048462 -0.181769
12     1.828686   1        23      0.126566 -0.064042
13     1.661174   1        24     -0.002530  0.056543
27     1.410672   1        25      0.197070  0.161715
28     1.407993   1        26      0.119480  0.162146
6      1.386631   1        27      0.157579  0.059726
22     1.208345   1        28      0.179581 -0.011061
3      1.081128   1        29      0.247938  0.034705
15     0.969031   1        30      0.182834  0.017067, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
175   Default  6585.000000   file          6585.000000   30.0000  30.0000
20    Default  3748.000000   type          3748.000000   29.0000  29.0000
310   Default  4014.000000   function      4014.000000   28.0000  28.0000
286   Default  3393.000000   test          3393.000000   27.0000  27.0000
559   Default  4391.000000   class         4391.000000   26.0000  26.0000
198   Default  4160.000000   name          4160.000000   25.0000  25.0000
73    Default  4061.000000   value         4061.000000   24.0000  24.0000
398   Default  3945.000000   object        3945.000000   23.0000  23.0000
430   Default  3051.000000   page          3051.000000   22.0000  22.0000
214   Default  3163.000000   table         3163.000000   21.0000  21.0000
21    Default  25035.000000  use           25035.000000  20.0000  20.0000
168   Default  4005.000000   datum         4005.000000   19.0000  19.0000
16    Default  3390.000000   project       3390.000000   18.0000  18.0000
91    Default  3786.000000   return        3786.000000   17.0000  17.0000
4     Default  8228.000000   code          8228.000000   16.0000  16.0000
196   Default  3768.000000   method        3768.000000   15.0000  15.0000
167   Default  2973.000000   database      2973.000000   14.0000  14.0000
538   Default  4490.000000   run           4490.000000   13.0000  13.0000
109   Default  3687.000000   user          3687.000000   12.0000  12.0000
5     Default  2172.000000   control       2172.000000   11.0000  11.0000
719   Default  3163.000000   list          3163.000000   10.0000  10.0000
1144  Default  2390.000000   string        2390.000000   9.0000   9.0000 
665   Default  2153.000000   java          2153.000000   8.0000   8.0000 
133   Default  4689.000000   new           4689.000000   7.0000   7.0000 
72    Default  3915.000000   time          3915.000000   6.0000   6.0000 
139   Default  2207.000000   select        2207.000000   5.0000   5.0000 
869   Default  2083.000000   question      2083.000000   4.0000   4.0000 
562   Default  2023.000000   library       2023.000000   3.0000   3.0000 
230   Def

### Model performance: LDA using Lemmatization

In [186]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_lemma.log_perplexity(corpus_lemma))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda_lemma = CoherenceModel(model=lda_model_lemma, texts=tokens_lemma, dictionary=dictionary_LDA_lemma, coherence='c_v')
coherence_lda_lemma = coherence_model_lda_lemma.get_coherence()
print('\nCoherence Score: ', coherence_lda_lemma)


Perplexity:  -13.673513147098323

Coherence Score:  0.43564814768786


## Fine-tuning the model

### Finding optimal number of topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    perplexity_values = []
    coherence_values = []
    model_list = []
    with tqdm(total=len(range(start, limit, step)), file=sys.stdout) as pbar:
        for num_topics in range(start, limit, step):
            #model = models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
            model = models.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=10, alpha=0.001, eta=0.001, random_state = 42)
            model_list.append(model)
            perplexity_values.append(model.log_perplexity(corpus))
            model_coherence = coherencemodel.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append(model_coherence.get_coherence())
            pbar.update(1)
            sleep(1)

    return perplexity_values, model_list, coherence_values

In [ ]:
# Can take a long time to run.
k_min = 2
k_max = 50
pas = 2
perplexity_values, model_list, coherence_values = compute_coherence_values(dictionary=dictionary_LDA_lemma, corpus=corpus_lemma,
                                                                           texts=tokens, start=k_min, limit=k_max, step=pas)

### Finding the dominant topic in each sentence

In [187]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [188]:
df_topic_sents_tag= format_topics_sentences(ldamodel=lda_model_lemma, corpus=corpus_lemma, texts=tokens_lemma)

# Format
df_dominant_topic = df_topic_sents_tag.reset_index()
df_dominant_topic.columns = ['Comment_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'TAGS', 'Text']

# Show
df_dominant_topic.head(10)

,Comment_No,Dominant_Topic,Topic_Perc_Contrib,TAGS,Text
0,0,18.0,0.2770,"code, can, use, write, one, make, will, way, like, need","[want, use, track, bar, change, form, opacity, code, decimal, tran, trackbar_value, opacity, tran, build, application, give, follow_error, implicitly, convert, type, decimal, double, try, use, tran, double, control, doesn_work, code, work_fine, past, vb_net, project]"
1,1,24.0,0.2951,"text, image, display, line, size, label, button, div, position, space","[absolutely_position_div, contain, several, child, one, relatively, position_div, use, percentage, base, width, child, div, collapse, width, internet_explorer, firefox_safari, use, pixel, width, work, parent, relatively, position, percentage, width, child, work, something, miss, easy, fix, besides, pixel, base, width, child, area, css, specification, cover]"
2,2,14.0,0.4198,"type, bit, use, compiler, int, can, will, system, value, integer","[explicit, cast, double, like, isn, necessary, double, tran, double, trackbar_value, identify, constant, sufficient, double, tran, trackbar_value, double, tran, trackbar_value]"
3,3,29.0,0.4895,"table, query, -pron-_would, select, column, name, row, field, index, update","[give, datetime, represent, person, birthday, calculate_age, year]"
4,4,24.0,0.2900,"text, image, display, line, size, label, button, div, position, space","[give, specific, datetime, value, display, relative, time, like, hour_ago, day_ago, month_ago]"
5,5,6.0,0.6696,"event, thread, service, document, date, day, tfs, assembly, web_service, start","[var, ts, new_timespan, datetime_utcnow, tick, dt, tick, double, delta, math, ab, ts, totalsecond, delta_return, ts, second, one, second_ago, ts, second, second_ago, delta_return, minute_ago, delta_return, ts, minute, minute_ago, delta_return, hour_ago, delta_return, ts, hour_hour, ago_delta, return, yesterday, delta_return, ts_day, day_ago, delta, int, month, convert_toint, math_floor, double, ts_day, return, month, one, month_ago, month_month, ago, int, year, convert_toint, math_floor, double, ts_day, return, year, one, year_ago, year, year_ago, suggestion, comment, way, improve, algorithm]"
6,6,9.0,0.3767,"window, run, application, server, can, use, machine, app, will, set","[standard, way, web_server, able, determine, user, timezone, within, web_page, perhaps, http_header, part, user_agent_string]"
7,7,3.0,0.3792,"address, color, socket, background, mark, sample, total, reader, sequence, bind","[difference, math_floor, math, truncate, net]"
8,8,22.0,0.3565,"view, model, map, load, product, controller, filter, create, viewstate, layer","[expose, linq_query, asmx_web_service, usually, business_tier, can, return, type, dataset_datatable, can, serialize, transport, asmx, can, linq_query, way, populate, type, dataset_datatable, via, linq_query, public_static, mydatatable, callmysproc, str, conn, mydatabasedatacontext, db, new, mydatabasedatacontext, conn, mydatatable, dt, new, mydatatable, execute, sproc, via, linq, var, query, dr, db, mysproc, asenumerable, select, dr, copy, linq_query, resultset, datatable, work, dt, query, copytodatatable, return, dt, can, get, result, set, linq_query, dataset_datatable, alternatively, linq_query, serializeable, can, expose, asmx_web_service]"
9,9,4.0,0.6091,"datum, database, use, access, can, db, store, application, need, sql","[store, binary_data, mysql]"


In [189]:
df_dominant_topic.tail(10)

,Comment_No,Dominant_Topic,Topic_Perc_Contrib,TAGS,Text
27402,27402,12.0,0.4391,"name, template, dll, message, path, import, root, target, status, system","[img_border, old_fashioned, img_border, src]"
27403,27403,24.0,0.6640,"text, image, display, line, size, label, button, div, position, space","[just, add, border, img_border, remove, border, image, link, trick]"
27404,27404,24.0,0.6705,"text, image, display, line, size, label, button, div, position, space","[code, use, border, example, img, href, mypic, gif, border, within, css, border, whatev, class, image]"
27405,27405,7.0,0.2246,"function, return, call, array, use, can, variable, end, value, will","[excel, way, gathering, attribute, build, function, re_willing, use, vb, color, relate_question, answer, http_www, cpearson, com, excel, color, aspx, example, form, site, sumcolor, function, color, base, analog, sum, sumif, function, allow, specify, separate, range_range, whose, color, index, examine, range, cell, whose, value, sum, two, range, function, sum, cell, whose, color, match, specify, value, example, follow, formula, sum, value, whose, fill, color_red, sumcolor, false]"
27406,27406,18.0,0.2618,"code, can, use, write, one, make, will, way, like, need","[simply, put, static_analysis, collect, information, base, source_code, dynamic_analysis, base, system, execution, often, use, instrumentation, advantage, dynamic_analysis, able, detect, dependency, possible, detect, static_analysis, ex, dynamic, dependency, use_reflection, dependency_injection, polymorphism, can, collect, temporal, information, deal, real, input, datum, static_analysis, difficult_impossible, know, file, will, pass, input, web, request, will, come, user, will, click, etc, disadvantage, dynamic_analysis, may, negatively, impact_performance, application, guarantee, full, coverage, source_code, run, base, user_interaction, automatic, test, resource, many, dynamic_analysis, tool, market, debugger, notorious, one, hand, still, academic, research, field, many, researcher, study, use, dynamic_analysis, better, understand, software, system, annual, workshop, dedicated, dependency, analysis]"
27407,27407,2.0,0.3723,"use, will, get, good, work, one, just, can, go, think","[occasionally, program, window, machine, go, crazy, just, hang, will, call, task_manager, hit, end, process, button, however, doesn, always, work, try, enough, time, will, usually, die, eventually, really, like, able, just, kill, immediately, linux, just, kill, guarantee, process, will, die, also, use, writing, batch_script, write, batch_script, programming, program, command, come, window, will, always, kill_process, free, third_party, app, fine, although, prefer, able, machine, sit, first, time]"
27408,27408,18.0,0.4027,"code, can, use, write, one, make, will, way, like, need","[taskkill_-pron-_be, myprocess, exe, force, know, pid, can, specify, taskkill, pid, lot, option, possible, just, type, taskkill, option, kill_process, child_process, may, useful]"
27409,27409,8.0,0.4246,"project, build, change, version, file, repository, svn, use, branch, add","[problem, isn, build, script, visual_studio, support, ansi, control, code, change, color]"
27410,27410,18.0,0.2432,"code, can, use, write, one, make, will, way, like, need","[use, switch, place, ssh_client, master, mode, connection, shar, e, multiple, option, place, ssh, master, mode, confirmation, require, slave, connection, accept, refer, description, controlmaster, ssh, config, detail, don, quite, see, answer, op, question, can, expand, bit, david]"
27411,27411,11.0,0.8746,"use, tool, can, net, visual_studio, also, application, project, run, build","[get, process_explorer, sysinternal, now, microsoft, process_explorer, window, sysinternal, microsoft, doc]"
